# Intro

Using Kaggle data from here:
https://www.kaggle.com/paultimothymooney/denver-crime-data/

85mb download contains several files, relevant to this project are crime.csv and offense_codes.csv

No need to join the two files, only additional info in offense_codes.csv is a plain English version of the OFFENSE_TYPE and OFFENSE_CATEGORY fields.  Example: "criminal-mischief-private" vs. "Criminal mischief to private property".  Will keep offense_codes.csv on hand for reference or if we need to display a plain English version of crime types in the future.

In [1]:
# Imports
import json
import pandas as pd
import requests


In [2]:
# Load csv data into Pandas Dataframe for EDA
crime = pd.read_csv('crime.csv')
crime.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [3]:
# crime.info()

In [4]:
# Cut down the dataset by removing traffic accidents: keep only (IS_CRIME == 1).
crime = crime[crime['IS_CRIME']==1]
# crime.info()

In [5]:
# Remove rows without latitude or longitude information
crime = crime.dropna(subset=['GEO_LON', 'GEO_LAT'])
# crime.info()

Remove columns we wont need for analysis.

Columns that will remain:
* OFFENSE_ID : unique identifier - concatenation of INCIDENT_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION (as 2-digits)
* OFFENSE_TYPE_ID : name of crime reported
* OFFENSE_CATEGORY_ID : category of crime reported
* REPORTED_DATE : date and time reported
* GEO_LON : longitude
* GEO_LAT : latitude
* DISTRICT_ID : district within Denver
* PRECINCT_ID : precinct within Denver
* NEIGHBORHOOD_ID : neighborhood within Denver


In [6]:
# crime.info()

In [7]:
crime = crime.drop(columns=['INCIDENT_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION',
                    'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'INCIDENT_ADDRESS', 
                    'GEO_X', 'GEO_Y', 'IS_CRIME', 'IS_TRAFFIC'])
crime.head(15)

,OFFENSE_ID,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,REPORTED_DATE,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID
0,2016376978521300,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,-104.809881,39.773188,5,521,montbello
1,20186000994239900,theft-other,larceny,1/29/2018 5:53:00 PM,-104.781434,39.785649,5,522,gateway-green-valley-ranch
2,20166003953230500,theft-items-from-vehicle,theft-from-motor-vehicle,4/26/2016 9:02:00 PM,-104.957381,39.663490,3,314,wellshire
3,201872333239900,theft-other,larceny,1/30/2018 10:29:00 PM,-104.941440,39.702698,3,312,belcaro
4,2017411405230300,theft-shoplift,larceny,6/23/2017 4:09:00 PM,-104.955370,39.717107,3,311,cherry-creek
5,201872837549900,traf-other,all-other-crimes,1/31/2018 1:29:00 AM,-104.961928,39.743149,6,622,city-park-west
6,20186001015230400,theft-parts-from-vehicle,theft-from-motor-vehicle,1/26/2018 12:24:00 PM,-105.025543,39.732790,1,122,villa-park
7,201870628570700,criminal-trespassing,all-other-crimes,1/30/2018 10:33:00 AM,-104.983794,39.723424,3,311,speer
9,20186001048230500,theft-items-from-vehicle,theft-from-motor-vehicle,1/31/2018 7:07:00 AM,-105.015451,39.757627,1,113,highland
10,201872390357200,drug-methampetamine-possess,drug-alcohol,1/30/2018 10:12:00 PM,-104.925197,39.678463,3,323,goldsmith


In [8]:
# crime.info()

Initial dataframe size after loading csv: 67.7 MB

Dataframe size after dropping non-crimes and columns: 26.1 MB

In [9]:
#Convert REPORTED_DATE to datetime and add columns for YEAR, MONTH, DAY
crime['REPORTED_DATE'] = pd.to_datetime(crime.REPORTED_DATE)
crime['YEAR'] = crime['REPORTED_DATE'].dt.year
crime['MONTH'] = crime['REPORTED_DATE'].dt.month
crime['DAY'] = crime['REPORTED_DATE'].dt.day
crime.head(15)

,OFFENSE_ID,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,REPORTED_DATE,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,YEAR,MONTH,DAY
0,2016376978521300,weapon-unlawful-discharge-of,all-other-crimes,2016-06-15 23:31:00,-104.809881,39.773188,5,521,montbello,2016,6,15
1,20186000994239900,theft-other,larceny,2018-01-29 17:53:00,-104.781434,39.785649,5,522,gateway-green-valley-ranch,2018,1,29
2,20166003953230500,theft-items-from-vehicle,theft-from-motor-vehicle,2016-04-26 21:02:00,-104.957381,39.663490,3,314,wellshire,2016,4,26
3,201872333239900,theft-other,larceny,2018-01-30 22:29:00,-104.941440,39.702698,3,312,belcaro,2018,1,30
4,2017411405230300,theft-shoplift,larceny,2017-06-23 16:09:00,-104.955370,39.717107,3,311,cherry-creek,2017,6,23
5,201872837549900,traf-other,all-other-crimes,2018-01-31 01:29:00,-104.961928,39.743149,6,622,city-park-west,2018,1,31
6,20186001015230400,theft-parts-from-vehicle,theft-from-motor-vehicle,2018-01-26 12:24:00,-105.025543,39.732790,1,122,villa-park,2018,1,26
7,201870628570700,criminal-trespassing,all-other-crimes,2018-01-30 10:33:00,-104.983794,39.723424,3,311,speer,2018,1,30
9,20186001048230500,theft-items-from-vehicle,theft-from-motor-vehicle,2018-01-31 07:07:00,-105.015451,39.757627,1,113,highland,2018,1,31
10,201872390357200,drug-methampetamine-possess,drug-alcohol,2018-01-30 22:12:00,-104.925197,39.678463,3,323,goldsmith,2018,1,30


In [10]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341777 entries, 0 to 467279
Data columns (total 12 columns):
OFFENSE_ID             341777 non-null int64
OFFENSE_TYPE_ID        341777 non-null object
OFFENSE_CATEGORY_ID    341777 non-null object
REPORTED_DATE          341777 non-null datetime64[ns]
GEO_LON                341777 non-null float64
GEO_LAT                341777 non-null float64
DISTRICT_ID            341777 non-null int64
PRECINCT_ID            341777 non-null int64
NEIGHBORHOOD_ID        341777 non-null object
YEAR                   341777 non-null int64
MONTH                  341777 non-null int64
DAY                    341777 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6), object(3)
memory usage: 33.9+ MB


# following is taken from Geoff Boeing's github pandas-to-geojson.ipynb
https://github.com/gboeing/urban-data-science/blob/3faf7e028d48cb03ddb999c5a910213c5384e7dc/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb

In [11]:
def df_to_geojson(df, properties, lat='GEO_LAT', lon='GEO_LON'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type': 'Feature',
                  'properties':{},
                  'geometry':{'type':'Point', 'coordinates':[]}}
        
        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        
        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
            
        # add this feature (aka converted dataframe row) to the list of features inside the dict
        geojson['features'].append(feature)
        
    return geojson

In [12]:
slice = crime.sample(n=2000)
useful_cols = ['OFFENSE_ID', 'OFFENSE_TYPE_ID', 'OFFENSE_CATEGORY_ID', 
                  'DISTRICT_ID', 'PRECINCT_ID', 'NEIGHBORHOOD_ID',
                  'YEAR', 'MONTH', 'DAY']

geojson_dict = df_to_geojson(slice, properties=useful_cols)
geojson_str = json.dumps(geojson_dict, indent=2)

In [13]:
output_filename = 'dataset.json'
with open(output_filename, 'w') as output_file:
    output_file.write(geojson_str)